In [6]:
pip install librosa pydub scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import shutil
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from pydub import AudioSegment

# Paths
source_dir = r"C:\Users\sagni\Downloads\Accent Detectection\archive (1)\recordings\recordings"
organized_dir = r"C:\Users\sagni\Downloads\Accent Detectection\organized_by_accent"

# Step 1: Organize files into folders by accent
os.makedirs(organized_dir, exist_ok=True)

for file in os.listdir(source_dir):
    if file.endswith(".mp3"):
        label = ''.join([c for c in file if not c.isdigit()]).replace(".mp3", "").strip()
        label_folder = os.path.join(organized_dir, label)
        os.makedirs(label_folder, exist_ok=True)
        shutil.copy(os.path.join(source_dir, file), os.path.join(label_folder, file))

print("✅ Files organized by accent.")

# Step 2: Convert .mp3 to .wav
for label_folder in os.listdir(organized_dir):
    folder_path = os.path.join(organized_dir, label_folder)
    for file in os.listdir(folder_path):
        if file.endswith(".mp3"):
            mp3_path = os.path.join(folder_path, file)
            wav_path = mp3_path.replace(".mp3", ".wav")
            try:
                sound = AudioSegment.from_mp3(mp3_path)
                sound.export(wav_path, format="wav")
            except Exception as e:
                print(f"❌ Error converting {mp3_path}: {e}")

print("✅ MP3s converted to WAV.")

# Step 3: Extract features
def extract_features(file_path):
    try:
        audio, sr = librosa.load(file_path, sr=None)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        return np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"❌ Error extracting features from {file_path}: {e}")
        return None

features = []
labels = []

for label_folder in os.listdir(organized_dir):
    folder_path = os.path.join(organized_dir, label_folder)
    for file in os.listdir(folder_path):
        if file.endswith(".wav"):
            file_path = os.path.join(folder_path, file)
            feature = extract_features(file_path)
            if feature is not None:
                features.append(feature)
                labels.append(label_folder)

print(f"✅ Extracted features from {len(features)} files.")

# Step 4: Train/test split and classification
if len(features) < 2:
    raise ValueError("❌ Not enough data to train a model.")

X = np.array(features)
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred))
print("🎯 Accuracy:", accuracy_score(y_test, y_pred))


✅ Files organized by accent.
✅ MP3s converted to WAV.
✅ Extracted features from 4276 files.

✅ Classification Report:
                 precision    recall  f1-score   support

      afrikaans       0.00      0.00      0.00         0
   afrikaans.mp       0.00      0.00      0.00         1
           akan       0.00      0.00      0.00         1
        akan.mp       0.00      0.00      0.00         0
       albanian       0.00      0.00      0.00         2
    albanian.mp       0.00      0.00      0.00         3
        amazigh       0.00      0.00      0.00         2
     amazigh.mp       0.00      0.00      0.00         0
        amharic       0.00      0.00      0.00         5
     amharic.mp       0.00      0.00      0.00         7
         arabic       0.00      0.00      0.00        24
      arabic.mp       0.00      0.00      0.00        23
       armenian       0.00      0.00      0.00         1
    armenian.mp       0.00      0.00      0.00         2
        ashanti       0.00